In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/ColumbiaMSCS/COMS4995_DL_for_CV/Project/Data/project_data.zip" "/content/project_data.zip"

In [ ]:
!mkdir -p "/content/data"

In [ ]:
!unzip -q "/content/project_data.zip" -d "/content/data"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import models
from torchvision.models import densenet201, DenseNet201_Weights
import numpy as np
import os
import shutil
import glob
import matplotlib.pyplot as plt
import random

In [ ]:
# Count of each class in the data
project_data = {}

project_data['spring'] = glob.glob('/content/data/project_data/spring/spring*')
project_data['summer'] = glob.glob('/content/data/project_data/summer/summer*')
project_data['fall'] = glob.glob('/content/data/project_data/fall/fall*')
project_data['winter'] = glob.glob('/content/data/project_data/winter/winter*')

print(f"count of spring images :  {len(project_data['spring'])}")
print(f"count of summer images :  {len(project_data['summer'])}")
print(f"count of fall images :  {len(project_data['fall'])}")
print(f"count of winter images :  {len(project_data['winter'])}")

count of spring images :  6000
count of summer images :  6000
count of fall images :  6000
count of winter images :  6000


In [ ]:
SOURCE = '/content/data/project_data'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = datasets.ImageFolder(root=SOURCE, transform=transform)

In [ ]:
# Use 4,200 images of each category for traning, 900 of each category for validation, 900 images of each category for testing
fall_samples = []
spring_samples = []
summer_samples = []
winter_samples = []

# Separate dataset into four label groups
for i in range(len(dataset)):
    _, label = dataset[i]
    if label == 0:
        fall_samples.append(i)
    elif label == 1:
        spring_samples.append(i)
    elif label == 2:
        summer_samples.append(i)
    else:
        winter_samples.append(i)

# Shuffle the samples
random.shuffle(fall_samples)
random.shuffle(spring_samples)
random.shuffle(summer_samples)
random.shuffle(winter_samples)

# Split the samples
train_indices = fall_samples[:4200] + spring_samples[:4200] + summer_samples[:4200] + winter_samples[:4200]
val_indices = fall_samples[4200:5100] + spring_samples[4200:5100] + summer_samples[4200:5100] + winter_samples[4200:5100]
test_indices = fall_samples[5100:6000] + spring_samples[5100:6000] + summer_samples[5100:6000] + winter_samples[5100:6000]

# Shuffle the indices
random.shuffle(train_indices)
random.shuffle(val_indices)
random.shuffle(test_indices)

# Create subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load pre-trained DenseNet201
model_densenet201 = densenet201(weights=DenseNet201_Weights.DEFAULT)

In [ ]:
# Change the final fully connected layer's output feature size to 4 so that
# it's suitable for softmax activation for classification on four seasons
model_densenet201.classifier.out_features = 4

In [ ]:
# Freeze all layers except the last modified layer
for name, param in model_densenet201.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [ ]:
# Use GPU
model_densenet201 = model_densenet201.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_densenet201.classifier.parameters(), lr=0.001, momentum=0.9)
softmax = nn.Softmax(dim=1)

In [ ]:
# Define training function
def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs, threshold):
    best_val_loss = float('inf')
    degrade_times = 0
    for epoch in range(epochs):
        train_loss = 0.0
        train_corrects = 0
        train_count = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(y_batch)
            pred = softmax(outputs)
            train_corrects += (torch.argmax(pred, dim=1) == y_batch).float().sum()
            train_count += y_batch.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_corrects / train_count

        val_loss = 0.0
        val_corrects = 0
        val_count = 0
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
                outputs = model(x_batch)
                loss = loss_fn(outputs, y_batch)
                val_loss += loss.item() * len(y_batch)
                pred = softmax(outputs)
                val_corrects += (torch.argmax(pred, dim=1) == y_batch).float().sum()
                val_count += y_batch.size(0)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects / val_count
        print(f'Epoch {epoch} Train Loss {train_loss:.4f} Train Accuracy {train_acc:.4f} Validation Loss {val_loss:.4f} Validation Accuracy {val_acc:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            degrade_times = 0
            # Save the model if it has the best validation loss so far
            torch.save(model.state_dict(), './best_model_densenet201.pth')
        else:
            degrade_times += 1
            # If the number of epochs where validation loss continuously increases
            # is larger than threshold, stop training the network to avoid overfitting
            if degrade_times > threshold:
                print(f'Early stopping at epoch {epoch}')
                break

In [ ]:
# Define evaluation function
def evaluate_model(model, test_loader):
    correct = 0
    count = 0
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            outputs = model(x_batch)
            pred = softmax(outputs)
            correct += (torch.argmax(pred, dim=1) == y_batch).float().sum()
            count += y_batch.size(0)
        test_acc = correct / count
    print(f'Accuracy on test set: {test_acc:.4f}')

In [ ]:
# Fine-tune the network on seasons image data
train_model(model_densenet201, train_loader, val_loader, loss_fn, optimizer, epochs=10, threshold=2)

Epoch 0 Train Loss 1.1944 Train Accuracy 0.6190 Validation Loss 0.7820 Validation Accuracy 0.7142
Epoch 1 Train Loss 0.7517 Train Accuracy 0.7145 Validation Loss 0.7178 Validation Accuracy 0.7381
Epoch 2 Train Loss 0.7001 Train Accuracy 0.7326 Validation Loss 0.6960 Validation Accuracy 0.7372
Epoch 3 Train Loss 0.6815 Train Accuracy 0.7389 Validation Loss 0.6686 Validation Accuracy 0.7461
Epoch 4 Train Loss 0.6724 Train Accuracy 0.7423 Validation Loss 0.6679 Validation Accuracy 0.7461
Epoch 5 Train Loss 0.6563 Train Accuracy 0.7462 Validation Loss 0.6659 Validation Accuracy 0.7519
Epoch 6 Train Loss 0.6543 Train Accuracy 0.7495 Validation Loss 0.6779 Validation Accuracy 0.7433
Epoch 7 Train Loss 0.6512 Train Accuracy 0.7491 Validation Loss 0.6629 Validation Accuracy 0.7483
Epoch 8 Train Loss 0.6391 Train Accuracy 0.7551 Validation Loss 0.6603 Validation Accuracy 0.7497
Epoch 9 Train Loss 0.6410 Train Accuracy 0.7516 Validation Loss 0.6640 Validation Accuracy 0.7511


In [ ]:
# Load the best model
model_densenet201.load_state_dict(torch.load('./best_model_densenet201.pth'))
# Evaluate on the test set
evaluate_model(model_densenet201, test_loader)

Accuracy on test set: 0.7444


In [ ]:
# Unfreeze all layers
for param in model_densenet201.parameters():
    param.requires_grad = True

In [ ]:
# Set the optimizer with a smaller learning rate
optimizer = optim.SGD(model_densenet201.parameters(), lr=0.0005, momentum=0.9)
# Continue fine-tuning the network
train_model(model_densenet201, train_loader, val_loader, loss_fn, optimizer, epochs=10, threshold=2)

Epoch 0 Train Loss 0.6204 Train Accuracy 0.7580 Validation Loss 0.5972 Validation Accuracy 0.7736
Epoch 1 Train Loss 0.4126 Train Accuracy 0.8455 Validation Loss 0.6015 Validation Accuracy 0.7672
Epoch 2 Train Loss 0.2861 Train Accuracy 0.9028 Validation Loss 0.6387 Validation Accuracy 0.7700
Epoch 3 Train Loss 0.2031 Train Accuracy 0.9376 Validation Loss 0.6711 Validation Accuracy 0.7694
Early stopping at epoch 3


In [ ]:
# Load the best model
model_densenet201.load_state_dict(torch.load('./best_model_densenet201.pth'))
# Evaluate on the test set
evaluate_model(model_densenet201, test_loader)

Accuracy on test set: 0.7672
